In [1]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

/sysadmin/wirawan/miniconda3/envs/bert2bert/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
model_name = "cahya/wav2vec2-large-xlsr-indonesian"
#model_name = "ayameRushia/wav2vec2-large-xlsr-indonesia-demo"
#model_name = "Galuh/wav2vec2-large-xlsr-indonesian-demo"

processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name) 
# model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [3]:
resampler = torchaudio.transforms.Resample(48_000, 16_000)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

In [4]:
test_dataset = load_dataset("common_voice", "id", split="test[:2%]")
test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:5], sampling_rate=16_000, return_tensors="pt", padding=True)

Couldn't find file locally at common_voice/common_voice.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/datasets/common_voice/common_voice.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/datasets/common_voice/common_voice.py.
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/id/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f)
Loading cached processed dataset at /root/.cache/huggingface/datasets/common_voice/id/6.1.0/0041e06ab061b91d0a23234a2221e87970a19cf3a81b20901474cffffeb7869f/cache-b6b4b4e0aff920be.arrow


In [5]:
with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

In [6]:
print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][:5])

Prediction: ['minggu depan kakak perempuan saya menikah', 'berbadi dahasa dairah dan bahasa ans inenja di beragsas akan temah kan permun jang menjadi bahasa inunisi', 'apa yang bisa sayah darikan kepadana', 'inilah dunia keci', 'nol']
Reference: ['Minggu depan kakak perempuan saya menikah.', 'Berbagai bahasa daerah dan bahasa asing menjadi bahasa serapan dan kemudian menjadi bahasa Indonesia.', 'apa yang bisa saya berikan kepadamu?', 'Inilah dunia kecil.', 'nol']
